In [2]:
%pip install langchain langchain-core langchain-community langchain-openai python-dotenv faiss-cpu -qU

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

In [4]:
load_dotenv()

model_name = "gpt-3.5-turbo-0125"
openai_api_key = os.environ["OPENAI_API_KEY"]
llm = ChatOpenAI(
    model_name = model_name,
    temperature = 0.1,
    max_tokens = 2000,
    openai_api_key = openai_api_key
)

In [5]:
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.evaluation.qa import QAEvalChain
from langchain_openai import OpenAIEmbeddings

In [6]:
loader = TextLoader("./worked.txt")
document = loader.load()

print(f"You have now {len(document)} document!")
print(f"Totally {len(document[0].page_content)} characters in the document!")

You have now 1 document!
Totally 75040 characters in the document!


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 3000, chunk_overlap = 400)
documents = text_splitter.split_documents(document)

In [8]:
embeddings = OpenAIEmbeddings(openai_api_key = openai_api_key)
document_search = FAISS.from_documents(documents, embeddings)

In [9]:
chain = RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff", retriever = document_search.as_retriever(), input_key = "question")

In [10]:
question_answers = [
    {
        "question": "which company sold the microcomputer kit that his friend built himself?",
        "answer": "Heathkit"
    },
    {
        "question": "what was the small city he talked about in the city that is the financial capital of USA?",
        "answer": "Yorkville, NY"
    }
]

In [11]:
predictions = chain.apply(question_answers)

C:\Users\asus\AppData\Local\Temp\ipykernel_3208\2904644452.py:1: LangChainDeprecationWarning: The method `Chain.apply` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~batch` instead.
  predictions = chain.apply(question_answers)


In [12]:
predictions

[{'question': 'which company sold the microcomputer kit that his friend built himself?',
  'answer': 'Heathkit',
  'result': 'The company that sold the microcomputer kit that his friend built himself was Heathkit.'},
 {'question': 'what was the small city he talked about in the city that is the financial capital of USA?',
  'answer': 'Yorkville, NY',
  'result': 'The small city mentioned in the context is Santa Cruz, located in California. The financial capital of the USA is New York City.'}]

In [14]:
evaluation_chain = QAEvalChain.from_llm(llm)

graded_output = evaluation_chain.evaluate(
    question_answers,
    predictions,
    question_key="question",
    prediction_key="result",
    answer_key="answer"
)

c:\000 - GenAI - OpenHouse\demonstrations\5-lc-advanced\env\Lib\site-packages\langchain_openai\chat_models\base.py:356: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


In [15]:
graded_output

[{'results': 'CORRECT'}, {'results': 'INCORRECT'}]